In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from pulp import * 

# List of suppliers
suppliers = ["PHX","AUS","GNV"]

# Dict for the number of units of supply for each supply node
supply = {
    "PHX" : 700,
    "AUS" : 200,
    "GNV" : 200
}

# List of demand nodes
demanders = ["LAX","DFW","ORD","ATL","JFK"]
# demanders = [1,2,3,4,5]

# Dict for the number of units of demand for each demand node
demand = {
    "LAX" : 200,
    "DFW" : 300,
    "ORD" : 200,
    "ATL" : 150,
    "JFK" : 250
}

# List of costs of each transportation path
costs = [ 
# -----demanders-----
# LAX DFW ORD ATL JFK
  [3,  3,  6,  7, -1], # PHX ---------------
  [7,  2, -1,  5, -1], # AUS    Suppliers
  [-1, 6, -1,  4,  7]  # GNV ---------------
]

In [3]:
# Create problem variable to contain data
prob = LpProblem("Shipping Distribution",LpMinimize)

# Create a list of tuples containing all the routes for transportation
routes = [(s,d) for s in suppliers for d in demanders]

# dict to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route",(suppliers,demanders),0,None,LpInteger)
print(route_vars)

# The objective function is added to prob first
temp = []
for (s,d) in routes:
    cost = costs[suppliers.index(s)][demanders.index(d)]
    if cost != -1:
        temp.append(route_vars[s][d]*cost)
prob += lpSum(temp), "Sum of Transporting Costs"

# The supply maximum constraints are added to prob for each supply node (supplier)
for s in suppliers:
    prob += lpSum([route_vars[s][d] for d in demanders]) <= supply[s], "Sum of Products out of Supplier %s"%s

# The demand minimum constraints are added to prob for each demand node (demander)
for d in demanders:
    prob += lpSum([route_vars[s][d] for s in supply]) >= demand[d], "Sum of Products into Demander %s"%d

{'PHX': {'LAX': Route_PHX_LAX, 'DFW': Route_PHX_DFW, 'ORD': Route_PHX_ORD, 'ATL': Route_PHX_ATL, 'JFK': Route_PHX_JFK}, 'AUS': {'LAX': Route_AUS_LAX, 'DFW': Route_AUS_DFW, 'ORD': Route_AUS_ORD, 'ATL': Route_AUS_ATL, 'JFK': Route_AUS_JFK}, 'GNV': {'LAX': Route_GNV_LAX, 'DFW': Route_GNV_DFW, 'ORD': Route_GNV_ORD, 'ATL': Route_GNV_ATL, 'JFK': Route_GNV_JFK}}


In [4]:
print(prob)

Shipping_Distribution:
MINIMIZE
5*Route_AUS_ATL + 2*Route_AUS_DFW + 7*Route_AUS_LAX + 4*Route_GNV_ATL + 6*Route_GNV_DFW + 7*Route_GNV_JFK + 7*Route_PHX_ATL + 3*Route_PHX_DFW + 3*Route_PHX_LAX + 6*Route_PHX_ORD + 0
SUBJECT TO
Sum_of_Products_out_of_Supplier_PHX: Route_PHX_ATL + Route_PHX_DFW
 + Route_PHX_JFK + Route_PHX_LAX + Route_PHX_ORD <= 700

Sum_of_Products_out_of_Supplier_AUS: Route_AUS_ATL + Route_AUS_DFW
 + Route_AUS_JFK + Route_AUS_LAX + Route_AUS_ORD <= 200

Sum_of_Products_out_of_Supplier_GNV: Route_GNV_ATL + Route_GNV_DFW
 + Route_GNV_JFK + Route_GNV_LAX + Route_GNV_ORD <= 200

Sum_of_Products_into_Demander_LAX: Route_AUS_LAX + Route_GNV_LAX
 + Route_PHX_LAX >= 200

Sum_of_Products_into_Demander_DFW: Route_AUS_DFW + Route_GNV_DFW
 + Route_PHX_DFW >= 300

Sum_of_Products_into_Demander_ORD: Route_AUS_ORD + Route_GNV_ORD
 + Route_PHX_ORD >= 200

Sum_of_Products_into_Demander_ATL: Route_AUS_ATL + Route_GNV_ATL
 + Route_PHX_ATL >= 150

Sum_of_Products_into_Demander_JFK: Route_AU